## Hodgkin-Huxley model with one channel in response to several voltage-clamps protocols
### Channel identity. ICG id: 487, ModelDB id: 123623

In [ ]:
import matplotlib as mpl
import numpy as np
import pandas as pd
import time

from matplotlib import pyplot as plt

## Voltage-clamp protocol

In [ ]:
prot = ['v_act','v_inact','v_deact','v_ap','v_ramp']
p = 1
protocols = \
pd.read_csv('~/Dropbox/Master/mackelab/code/lfi-models/lfimodels/channelomics/protocols/k_channels/'+prot[p]+'.dat',sep='\t')

protocols = protocols.drop(protocols.columns[-1],axis=1)

# extract voltages and time
protocols1 = protocols.values

V = protocols1[:,1:].T
t = protocols1[:,0]   # attention: time step varies, but we will assume that it is constant
tstep = np.mean(np.diff(t)) # ms

## Parameters and kinetics

In [ ]:
# parameters
celsius = 37    # original temperature
temp = 37       # reference temperature
q10 = 2.3       # temperature sensitivity
tadj = q10**((celsius - temp)/10)

# length and diameter equal to 20 um
A_soma = np.pi*((20.*1e-4)**2)  # cm2

C = 1 # uF/cm2

g_L = 1e-1 # mS/cm2
E_L = -70 # not sure which one is used in Podlaski et al. 2017, mV

gbar_K = 5.0 # mS/cm2
E_K = -90. # mV

noise_std = 0.001

###############################################################################
# kinetics
power = 4
vt = -63
param3 = 0.032
param4 = 15.
param5 = 5.
param6 = 0.5
param7 = 10.
param8 = 40.

# gt
# np.array([[4, -63, -0.032, 15, 5, 0.5, 10, 40]])

def alpha_n(x):
    v1 = x - vt - param4
    return param3*efun(-v1, param=param5)

def beta_n(x):
    v1 = x - vt - param7
    return param6*np.exp(-v1/param8)

def efun(z, param=1.):
    ind = np.abs(z) < 1e-4
    efun1 = z * 1.
    efun1[ind] = 1 - z[ind] / 2
    efun1[~ind] = z[~ind] / (np.exp(z[~ind] / param) - 1)
    return efun1

def n_inf(x):
    return alpha_n(x) / (alpha_n(x) + beta_n(x))

## Voltage-clamp simulation

In [ ]:
t_sim = time.time()

n = np.zeros_like(V)

n[:, 0] = n_inf(V[:, 0])

len_n = len(n[:,0])

for i in range(1, t.shape[0]):
    # simple one step euler
    ni = n[:, i-1]
    n[:, i] = ni + tstep * tadj * (alpha_n(V[:, i-1]) * (1 - ni) - beta_n(V[:, i-1]) * ni 
                                  + noise_std * np.random.randn(len_n) / (tstep**0.5))
print(time.time() - t_sim)


fact_inward = 1 # inward current pre-multiplied by -1 (see page 15 of Podlasky et al. 2017)

I_K = fact_inward * tadj * gbar_K * n**power * (V - E_K)
I_K = I_K/np.max(I_K)
# I_L = tadj*g_L*(V-E_L)
# I_tot = (I_K + I_L)/np.max(I_K + I_L)

## Plot

In [ ]:
num_levels = len(V[:,0])
cm1 = mpl.cm.viridis
col1 = [cm1(1.*i/num_levels) for i in range(num_levels)]

plt.figure(figsize=(18, 10))
gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
ax = plt.subplot(gs[0])
for i in range(num_levels):
    plt.plot(t, I_K[i,:], color = col1[i])
plt.ylabel('norm. current')
ax.set_xticks([])
# ax.set_yticks([0, 1.001*np.max(I_K)])

ax = plt.subplot(gs[1])
for i in range(num_levels):
    plt.plot(t, V[i, :],'r')
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')

duration = np.max(t)
ax.set_xticks([0, duration/2, duration])
ax.set_yticks([0, np.max(V)])

plt.show()

## Checking model implementation against it (attention: discrepancy in temporal resolution)

In [ ]:
from lfimodels.channelomics.ChannelSingle import ChannelSingle
n_params = 8
cython = False
m = ChannelSingle(channel_type='kd', n_params=n_params, cython=cython)

import numpy as np
t_sim = time.time()

out = m.gen_single(np.array([4, -63, 0.032, 15, 5, 0.5, 10, 40]))
print(time.time() - t_sim)

import matplotlib.pyplot as plt
%matplotlib inline

K2 = out[prot[p]]['data']
t2 = out[prot[p]]['time']

In [ ]:
num_levels = len(V[:,0])
cm1 = mpl.cm.viridis
col1 = [cm1(1.*i/num_levels) for i in range(num_levels)]

plt.figure(figsize=(18, 10))
gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[4, 1])
ax = plt.subplot(gs[0])
for i in range(num_levels):
    plt.plot(t2, K2[i,], color = col1[i])
plt.ylabel('norm. current')
ax.set_xticks([])
ax.set_yticks([0, 1.001*np.max(I_K)])

ax = plt.subplot(gs[1])
for i in range(num_levels):
    plt.plot(t, V[i,],'r')
plt.xlabel('time (ms)')
plt.ylabel('voltage (mV)')

duration = np.max(t)
ax.set_xticks([0, duration/2, duration])
ax.set_yticks([0, np.max(V)])

plt.show()